# Proposed Steps for CALSIM postprocessing

In [ ]:
import os
import requests
import json
import base64
import warnings
warnings.filterwarnings('ignore')

from ipyleaflet import (Map, Marker, basemaps, GeoJSON, AwesomeIcon, LayersControl,
                        FullScreenControl, LegendControl, WidgetControl, Icon, LayerGroup, Popup, MeasureControl)
from ipywidgets import Layout, Label, VBox, HTML
import geopandas as gpd
from myst_nb import glue

current_dir = os.getcwd()
in_notebook = False
if os.path.split(current_dir)[1] == "proposed_steps":
    os.chdir("..")
    os.chdir("..")
    os.chdir("..")
    in_notebook = True

from python_functions.maps import (add_geojson_points_markers, make_svg_and_geojson,
                                   query_rest_api, make_reservoir_polygon,
                                   make_ag_surface_and_gw_polygon, make_ag_gw_polygon,
                                   make_urban_gw_and_surface_polygon, make_conv_circle,
                                   make_gaged_conv_circle, make_conv_channel, make_gaged_conv_channel,
                                   external_polygon, arcs_geojson_and_legend)
if in_notebook:
    os.chdir(current_dir)

# We'll make a dictionary of paths where DropBox is mounted on everyone's
# machine by user login

db_dir_dict = {
    "JoseTomasDiazCasanue": os.path.join(
        r"C:\Users\JoseTomasDiazCasanue\LWA Dropbox"
    )
}

if os.getlogin() in db_dir_dict.keys():
    db_dir = db_dir_dict[os.getlogin()]
else:
    print("You need to add your user and the path where DropBox is mounted on your machine"
          "to the db_dir_dict dictionary")

# Path to calsim GIS
calsim_gis_dir = os.path.join(
    db_dir,
    "01_Project-Teams",
    "00233.24-DMC-Studies",
    "Far-Field-Dilution-Study",
    "documents",
    "work-plan",
    "maps",
    "GIS")

# Path to the shapefile with the NVRRWP discharge point
nvrrwp_dis_path = os.path.join(
    calsim_gis_dir,
    "NVRRWP_discharge_4326.shp"
)

# Function to add nodes as markers to maps

# This function makes the geojson dictionaries and icons for plotting the demand nodes

nvrrwp_dis_gdf = gpd.read_file(nvrrwp_dis_path)

center = (37.3, -121.1)

nvrrwp_dis_loc = (nvrrwp_dis_gdf.geometry.y.values[0],nvrrwp_dis_gdf.geometry.x.values[0])

# Let's query reservoirs
reservoirs_url = "https://gis.water.ca.gov/arcgis/rest/services/InlandWaters/NHD_Major_Lakes_and_Reservoirs/FeatureServer/0/query"
reservoirs_gdf = query_rest_api("gnis_name IN ('San Luis Reservoir', 'O''Neill Forebay')", reservoirs_url)

# Let's convert to geojsons
reservoirs_geojson = reservoirs_gdf.to_geo_dict()

html = HTML(""" """)
html.layout.margin = "0px 5px 5px 5px"
hover_control = WidgetControl(widget=html, position="bottomleft")
def update_html(feature, **kwargs):
    html.value = """
        <h3 style="font-size: 12px;"><b>{}</b></h3>
    """.format(
        feature["properties"]["gnis_name"]
    )


reservoirs_geojson = GeoJSON(
    data=reservoirs_geojson,
    style={
        'weight': 2,
        "color": "#a6cee3",
        "fillColor": "#a6cee3",
        'fillOpacity': 1
    },
    hover_style={"fillColor": "yellow"},
    name="Reservoirs"
)

reservoirs_geojson.on_hover(update_html)

# Define a custom icon (URL of your custom icon)
nvrrwp_dis_icon_url = "https://upload.wikimedia.org/wikipedia/commons/e/ec/RedDot.svg"

# Create an Icon object
nvrrwp_dis_icon = Icon(icon_url=nvrrwp_dis_icon_url, icon_size=[30, 30])

# Let's import geojson with nodes
nodes_geojson_path = os.path.join(
    calsim_gis_dir,
    "Calsim_Nodes.geojson"

)

nodes_gdf = gpd.read_file(nodes_geojson_path)

# Nodes representing reservoirs
reservoir_nodes_gdf = nodes_gdf.loc[
    nodes_gdf["NodeDescription"] == "storage- reservoir or lake"
].reset_index(drop=True)

reservoir_nodes_geojson = reservoir_nodes_gdf.to_geo_dict()

# Now, let's make the icon that will represent reservoirs
size = 50

reservoir_svg =f'''
    <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 {size} {size}" width="{size/2}" height="{size/2}">
    {make_reservoir_polygon(size)}
    </svg>
    '''


# Encode the SVG into a data URL
reservoir_icon_url = "data:image/svg+xml;base64," + base64.b64encode(reservoir_svg.encode('utf-8')).decode('utf-8')

reservoir_icon = Icon(icon_url=reservoir_icon_url, icon_size=[size/2, size/2], icon_anchor=[size/4, size/4])



# Nodes representing agricultural demand units that use both surface water and groundwater

# Let's make the icon now

agg_surface_and_gw_nodes_svg = make_ag_surface_and_gw_polygon(size)

agg_surface_and_gw_nodes_geojson, agg_surface_and_gw_node_icon = make_svg_and_geojson(
    "demand-agricultural-surface&groundwater", 
    nodes_gdf, agg_surface_and_gw_nodes_svg, size)

# Nodes representing agricultural demand units that use only groundwater

# Let's make the icon now

agg_gw_nodes_svg = make_ag_gw_polygon(size)

agg_gw_nodes_geojson, agg_gw_node_icon = make_svg_and_geojson(
    "demand-agricultural-groundwater", 
    nodes_gdf, agg_gw_nodes_svg, size)

# Nodes representing urban demand units that use groundwater and surface water

# Let's make the icon now

urban_gw_and_surface_nodes_svg = make_urban_gw_and_surface_polygon(size)

urban_gw_and_surface_nodes_geojson, urban_gw_and_surface_node_icon = make_svg_and_geojson(
    "demand-urban-surface&groundwater", 
    nodes_gdf, urban_gw_and_surface_nodes_svg, size)

# Nodes representing conveyance canals

# Let's make the icon now

conv_circle_svg = make_conv_circle(size)

conv_circle_nodes_geojson, conv_circle_node_icon = make_svg_and_geojson(
    "conveyance-canal", 
    nodes_gdf, conv_circle_svg, size)

# Nodes representing conveyance canals
conv_circle_svg = make_conv_circle(size)
conv_circle_nodes_geojson, conv_circle_node_icon = make_svg_and_geojson(
    "conveyance-canal", 
    nodes_gdf, conv_circle_svg, size)

# Nodes representing gaged conveyance canals

# Let's make the icon now
gaged_conv_circle_svg = make_gaged_conv_circle(size)

gaged_conv_circle_nodes_geojson, gaged_conv_circle_node_icon = make_svg_and_geojson(
    "conveyance-canal-streamgage", 
    nodes_gdf, gaged_conv_circle_svg, size)

# Nodes representing conveyance channels

# Let's make the icon now
conv_channel_nodes_svg = make_conv_channel(size)

conv_channel_nodes_geojson, conv_channel_nodes_icon = make_svg_and_geojson(
    "conveyance-channel", 
    nodes_gdf, conv_channel_nodes_svg, size)

legend = HTML(
    value=f"""
    <b>Legend:</b><br>
    <svg width="200" height="27">
        <image href="{nvrrwp_dis_icon_url}" x="5" y="2.5" height="20px" width="20px"/>
        <text x="50" y="12" fill="black">NVRRWP</text>
        <text x="50" y="25" fill="black">discharge location</text>
    </svg><br>
    <svg width="200" height="20">
            <polygon points="15,5 9,17 21,17" style="fill:#a6cee3; stroke:#1f78b4; stroke-width:5" />
            <text x="50" y="15" text-anchor="left" fill="black">Reservoirs</text>
        </svg> <br>
    <svg width="200" height="37">
        <rect x="0" y="10" width="30" height="15" style="fill:#4caf50; stroke:#003300; stroke-width:4;"/>
        <rect x="4" y="14" width="22" height="7" style="fill:#4caf50; stroke:#003300; stroke-width:2;"/>
        <text x="50" y="10" text-anchor="left" fill="black">Agricultural demand units</text>
        <text x="50" y="22" fill="black">using both surface water</text>
        <text x="50" y="33" fill="black">and groundwater</text>
    </svg> <br>
    <svg width="200" height="30">
        <rect x="2" y="8" width="28" height="15" style="fill:#4caf50; stroke:#003300; stroke-width:4; stroke-dasharray: 5, 5;"/>
        <text x="50" y="13" text-anchor="left" fill="black">Agricultural demand units</text>
        <text x="50" y="27" fill="black">using only groundwater</text>
    </svg> <br>
    <svg width="200" height="37">
        <rect x="0" y="10" width="30" height="15" style="fill:#969696; stroke:#003300; stroke-width:4;"/>
        <rect x="4" y="14" width="22" height="7" style="fill:#969696; stroke:#003300; stroke-width:2;"/>
        <text x="50" y="10" text-anchor="left" fill="black">Urban demand units</text>
        <text x="50" y="22" fill="black">using both surface water</text>
        <text x="50" y="33" fill="black">and groundwater</text>
    </svg> <br>
    <svg width="200" height="25">
            <circle cx="16" cy="12" r="9" style="fill:white; stroke:gray; stroke-width:5;" />
            <text x="50" y="15" text-anchor="left" fill="black">Conveyance canals</text>
    </svg> <br>
    <svg width="200" height="25">
        <circle cx="16" cy="12" r="9" style="fill:gray; stroke:gray; stroke-width:5;" />
        <text x="50" y="11" fill="black">Gaged conveyance</text>
        <text x="50" y="22" fill="black">canals</text>
    </svg> <br>
    <svg width="200" height="25">
            <circle cx="16" cy="12" r="9" style="fill:white; stroke:blue; stroke-width:5;" />
            <text x="50" y="15" text-anchor="left" fill="black">Conveyance channels</text>
    </svg> <br>
    """,
    placeholder="Legend",
    description="",
)

# Nodes representing gaged conveyance channels

# Let's make the icon now
gaged_conv_channel_nodes_svg = make_gaged_conv_channel(size)

gaged_conv_channel_nodes_geojson, gaged_conv_channel_nodes_icon = make_svg_and_geojson(
    "conveyance-channel-streamgage", 
    nodes_gdf, gaged_conv_channel_nodes_svg, size)

legend.value = legend.value + """<svg width="200" height="25">
            <circle cx="16" cy="12" r="9" style="fill:gray; stroke:blue; stroke-width:5;" />
            <text x="50" y="15" text-anchor="left" fill="black">Gaged Conveyance channels</text>
    </svg> <br>"""

# Nodes representing external demand units

# Let's make the icon now
external_nodes_svg = external_polygon(size)

external_nodes_geojson, external_nodes_icon = make_svg_and_geojson(
    "external-unit", 
    nodes_gdf, external_nodes_svg, size)

legend.value = legend.value + """<svg width="200" height="30">
    <rect x="2" y="8" width="28" height="15" style="fill:orange; stroke:#003300; stroke-width:4;"/>
    <text x="50" y="13" text-anchor="left" fill="black">External</text>
    <text x="50" y="26" fill="black">demand units</text>
    </svg> <br>"""

# Let's import arcs now
arcs_geojson_path = os.path.join(
    calsim_gis_dir,
    "Calsim_Arcs.geojson"

)

arcs_gdf = gpd.read_file(arcs_geojson_path)
arcs_gdf["x_centroid"] = arcs_gdf.centroid.x
arcs_gdf["y_centroid"] = arcs_gdf.centroid.y

condition = arcs_gdf["Sub_Type"] == "Bypass"

# Let's start with bypasses
bypass_geojson, legend = arcs_geojson_and_legend(arcs_gdf, condition,
                                                 "black", legend, "Bypass", "5, 5")

# Let's do canals now
condition_canal = (
        arcs_gdf["Type"] == "Channel"
) & (
        arcs_gdf["Sub_Type"] == "Canal"
)
canal_geojson, legend = arcs_geojson_and_legend(arcs_gdf, condition_canal,
                                                "grey", legend, "Canals", "")

# Let's do streams
condition_streams = (
        arcs_gdf["Type"] == "Channel"
) & (
        arcs_gdf["Sub_Type"] == "Stream"
)
streams_geojson, legend = arcs_geojson_and_legend(arcs_gdf, condition_streams,
                                                "blue", legend, "Streams", "")

# Diversions
condition_diversions = arcs_gdf["Type"] == "Diversion"

diversions_geojson, legend = arcs_geojson_and_legend(arcs_gdf, condition_diversions,
                                                "red", legend, "Diversions", "")

# Rim watershed inflows
condition_inflow = arcs_gdf["Type"] == "Inflow"

inflow_geojson, legend = arcs_geojson_and_legend(arcs_gdf, condition_inflow,
                                                "#a6cee3", legend, "Inflows", "")

# Return flows
condition_return = arcs_gdf["Type"] == "Return"

return_geojson, legend = arcs_geojson_and_legend(arcs_gdf, condition_return,
                                                "green", legend, "Return flows", "")

# Runoff
condition_runoff = arcs_gdf["Type"] == "Surface Runoff"

runoff_geojson, legend = arcs_geojson_and_legend(arcs_gdf, condition_runoff,
                                                "#bc80bd", legend, "Runoff", "")

m = Map(center=center, zoom=10,layout=Layout(width="80%",height="1000px"), basemap=basemaps.Esri.WorldGrayCanvas)
nvrrwp_dis_marker = Marker(location=nvrrwp_dis_loc, draggable=False, icon=nvrrwp_dis_icon, name="NVRRWP Discharge Location")

m.add(nvrrwp_dis_marker)
m.add(FullScreenControl(position="topleft"))
m.add(hover_control)

m = add_geojson_points_markers(m, reservoir_nodes_geojson, reservoir_icon)

# Let's add nodes representing agricultural demands met with both surface water and groundwater

m = add_geojson_points_markers(m, agg_surface_and_gw_nodes_geojson, agg_surface_and_gw_node_icon)

# Let's add nodes representing agricultural demands met only with groundwater
m = add_geojson_points_markers(m, agg_gw_nodes_geojson, agg_gw_node_icon)

# Let's add nodes representing urban demands met both with groundwater and surface water
m = add_geojson_points_markers(m, urban_gw_and_surface_nodes_geojson, urban_gw_and_surface_node_icon)

# Let's add nodes representing conveyance canals
m = add_geojson_points_markers(m, conv_circle_nodes_geojson, conv_circle_node_icon)

# Let's add nodes representing gaged conveyance canals
m = add_geojson_points_markers(m, gaged_conv_circle_nodes_geojson, gaged_conv_circle_node_icon)

# Let's add nodes representing conveyance channels
m = add_geojson_points_markers(m, conv_channel_nodes_geojson, conv_channel_nodes_icon)

# Let's add nodes representing gaged conveyance channels
m = add_geojson_points_markers(m, gaged_conv_channel_nodes_geojson, gaged_conv_channel_nodes_icon)

# Let's add nodes representing external demand units
m = add_geojson_points_markers(m, external_nodes_geojson, external_nodes_icon)

# Pop ups for lines
def geojson_onclick_handler(event=None, id=None, properties=None, **args):
    global selected_layer
    if properties is None:
        return
    name = properties["Arc_ID"]
    x = properties['x_centroid']
    y = properties['y_centroid']
    message1 = HTML()
    message1.value = name
    popup = Popup(
            location=(y,x),
            child=message1,
            close_button=True,
            auto_close=False,
            close_on_escape_key=False
        )
    m.add(popup)
# Let's add arcs representing bypasses
bypass_geojson.on_click(geojson_onclick_handler)
m.add(bypass_geojson)

# Let's add arcs representing canals
canal_geojson.on_click(geojson_onclick_handler)
m.add(canal_geojson)

# Let's add arcs representing streams
streams_geojson.on_click(geojson_onclick_handler)
m.add(streams_geojson)

# Let's add arcs representing streams
diversions_geojson.on_click(geojson_onclick_handler)
m.add(diversions_geojson)

# Let's add arcs representing inflows
inflow_geojson.on_click(geojson_onclick_handler)
m.add(inflow_geojson)

# Let's add arcs representing return flows
return_geojson.on_click(geojson_onclick_handler)
m.add(return_geojson)

# Let's add arcs representing runoff
runoff_geojson.on_click(geojson_onclick_handler)
m.add(runoff_geojson)

legend_control = WidgetControl(widget=legend, position="topright")
m.add(legend_control)
control = LayersControl(position='topleft')
m.add(control)

measure = MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    primary_length_unit = 'kilometers'
)
m.add(measure)

# Combine the map and any additional content in a VBox layout
vbox = VBox([m])

# Display the VBox with the map

glue("CALSIM_diagram", vbox, display=False)

```{glue:figure} CALSIM_diagram
:name: "calsim_map"

CALSIM Diagram
```

## Modelling Approach

The suggested modelling approach consists of executing a series of monthly mass balances for all the CALSIM nodes between the NVRRWP discharge and the outlet of San Luis Reservoir. Additional nodes might be incorporated later when DPWD sends us precise locations and flows of diversions between the NVWRRP discharge point and San Luis Reservoir that are not currently represented in CALSIM or are aggregated at the CALSIM nodes level. CALSIM nodes between the NVWRRP discharge point and San Luis Reservoir can be classified in two types: conveyance and reservoir. The nodes are presented in {numref}`system_nodes`

```{list-table} System Nodes
:header-rows: 1
:name: system_nodes

* - Node
  - Type
  - Inflows
  - Outflows
* - DMC039
  - Conveyance Canal
  - C_DMC034 <br> NVRRWP
  - C_DMC039 <br> D_DMC039_DMCLOS
* - DMC044
  - Conveyance Canal
  - C_DMC039
  - D_DMC044_71_PA5 <br> D_DMC044_71_PA4 <br> C_DMC044 <br> D_DMC044_DMCLOS
* - DMC049
  - Conveyance Canal
  - C_DMC044
  - C_DMC049 <br> D_DMC049_DMCLOS
* - DMC054
  - Conveyance Canal
  - C_DMC049
  - D_DMC054_NMW004 <br> C_DMC054 <br> D_DMC054_DMCLOS
* - DMC058
  - Conveyance Canal
  - C_DMC054
  - C_DMC058 <br> D_DMC058_DMCLOS
* - DMC064
  - Conveyance Canal
  - C_DMC058
  - D_DMC064_71_PA6 <br> C_DMC064 <br> D_DMC064_DMCLOS
* - DMC070
  - Conveyance Canal
  - C_DMC064 <br> D_CAA069_DMC070
  - D_DMC070_VLW008 <br> D_DMC070_73_PA1 <br> D_DMC070_CAA069 <br> C_DMC070 <br> D_DMC070_DMCLOS
* - CAA069
  - Conveyance Canal
  - C_CAA067 <br> C_SLUIS <br> D_DMC070_CAA069
  - D_CAA069_DMC070 <br> C_CAA069 <br> D_CAA069_SLUIS
* - SLUIS
  - Reservoir
  - D_CAA069_SLUIS <br> I_SLUIS <br>
  - D_SLUIS_PCH000 <br> C_SLUIS <br> E_SLUIS_CVP <br> E_SLUIS_SWP
  
```
For each node and time step, we will have a flow balance, which is necessary to characterize all the inflows and outflows. The first flow balance will be done for node DMC039, as shown in [text](#flow_balance_DMC039). Flow at node DMC034  will be adjusted to account for additional diversions at nodes DMC030 and DMC34 resulting from NVWRRP discharge.



```{math}
:label: flow_balance_DMC039

Q_{C\_DMC034}^{t} + Q_{NVRRWP}^{t} = Q_{DMC039}^{t} + Q_{D\_DMC039\_DMCLOS}^{t}

```

Where:
- {math}`t`: Simulation timestep.
- {math}`Q_{C\_DMC034}^{t}`: Outflow at node DMC034 for timestep {math}`t`
- {math}`Q_{NVRRWP}^{t}`: NVRRWP discharge for timestep {math}`t`
- {math}`Q_{C_DMC039}^{t}`: Outflow at node DMC039 for timestep {math}`t`
- {math}`Q_{D\_DMC039\_DMCLOS}^{t}`: Losses at node DMC039 for timestep {math}`t`

We can rearrange terms in [text](#flow_balance_DMC039) to solve for {math}`Q_{C_DMC039}^{t}`, which is the flow component that we need to update to account for NVRRWP discharge, as shown in [text](#DMC039_outflow)

```{math}
:label: DMC039_outflow

Q_{C\_DMC039}^{t} = Q_{C\_DMC034}^{t} + Q_{NVRRWP}^{t} - Q_{D\_DMC039\_DMCLOS}^{t}

```
The fraction of NVRRWP discharge entering node DMC039 can be defined as shown in [text](f0)   
```{math}
:label: f0

f_{0}^{t} = \frac{Q_{NVRRWP}^{t}}{Q_{NVRRWP}^{t}+Q_{C\_DMC034}^{t}}

```

Where:
- {math}`f_{0}^{t}`: Fraction of NVWRRP water at discharge point for timestep {math}`t`

This assumes that there is full mixing between the NVRRWP discharge and node DMC039. If we stick with CALSIM node locations, this seems like a reasonable assumption, as node DMC039 is located roughly 2,100 m downstream of the NVWRRP discharge point. If we find intermediate diversions between the NVWRRP discharge point and CALSIM node DMC039, this assumption will be revisited.

In addition to the flow balances, for each node and timestep we will have a mass balance, which is necessary to calculate the fractions of NVWRRP water. The first mass balance will be done in node DMC039, as shown in [text](#DMC039_mass_balance).

```{math}
:label: DMC039_mass_balance

(Q_{C\_DMC034}^{t}+Q_{NVRRWP}^{t})*f_{0}^{t} = Q_{C\_DMC039}^{t}*f_{DMC039}^{t}

```
Where:
- {math}`f_{DMC039}^{t}`: Fraction of NVWRRP water at node DMC039 for timestep {math}`t`

{math}`f_{DMC039}^{t}` is calculated rearranging [text](#DMC039_mass_balance) into [text](#f_DMC039).

```{math}
:label: f_DMC039

f_{DMC039}^{t}=\frac{Q_{C\_DMC034}^{t}+Q_{NVRRWP}^{t}}{Q_{C\_DMC039}^{t}}*f_{0}^{t}

```

The outflows of each node are calculated from the flow balances [text](#DMC044_outflow), [text](#DMC049_outflow), [text](#DMC054_outflow), [text](#DMC058_outflow), [text](#DMC064_outflow), [text](#DMC070_outflow), [text](#CAA069_flow_balance), and [text](#SLUIS_flow_balance_fd) using the same method as [text](#flow_balance_DMC039) and [text](#DMC039_outflow). Fractions of NVWRRP are calculated from the mass balances [text](#DMC044_mass_balance), [text](#DMC049_mass_balance), [text](#DMC054_mass_balance), [text](#DMC058_mass_balance), [text](#DMC064_mass_balance), [text](#DMC070_mass_balance), [text](#CAA069_mass_balance), and [text](#SLUIS_mass_balance_fd). Diversions at nodes DMC044 and DMC064 corresponding to DPWD's CVP contract deliveries ({math}`Q_{D\_DMC044\_71\_PA5}^{t}` and {math}`Q_{D\_DMC064\_71\_PA6}^{t}`) will be updated to account for additional diverted flows derived from NVWRRP discharge and storage at San Luis Reservoir.

```{math}
:label: DMC044_outflow

Q_{C\_DMC044}^{t} = Q_{C\_DMC039}^{t} - Q_{D\_DMC044\_DMCLOS}^{t} - Q_{D\_DMC044\_71\_PA5}^{t} - Q_{D\_DMC044\_71\_PA4}^{t}

```

Where:
- {math}`Q_{C\_DMC044}^{t}`: Outflow at node DMC044 for timestep {math}`t`
- {math}`Q_{D\_DMC044\_DMCLOS}^{t}`: Losses at node DMC044 for timestep {math}`t`
- {math}`Q_{D\_DMC044\_71\_PA5}^{t}`: Diversions to Del Puerto WD, Salado WD, Orestimba WD, and Sunflower WD for timestep {math}`t`
- {math}`Q_{D\_DMC044\_71\_PA4}^{t}`: Diversion to Patterson ID for timestep {math}`t`

The fraction of NVWRRP discharge at node DMC044 ({math}`f_{DMC044}^{t}`) is calculated using the mass balance shown in [text](#DMC044_mass_balance)
```{math}
:label: DMC044_mass_balance

Q_{C\_DMC039}^{t}*f_{DMC039}^{t} = (Q_{C\_DMC044}^{t} + Q_{D\_DMC044\_71\_PA4}^{t} + Q_{D\_DMC044\_71\_PA5}^{t})*f_{DMC044}^{t}

```
Where:
- {math}`f_{DMC044}^{t}`: Fraction of NVWRRP water at node DMC044 for timestep {math}`t`

{math}`f_{DMC044}^{t}` is calculated rearranging [text](#DMC044_mass_balance) into [text](#f_DMC044)

```{math}
:label: f_DMC044

f_{DMC044}^{t} = \frac{Q_{C\_DMC039}^{t}*f_{DMC039}^{t}}{Q_{C\_DMC044}^{t} + Q_{D\_DMC044\_71\_PA4}^{t} + Q_{D\_DMC044\_71\_PA5}^{t}}

```

The outflow for node DMC049 is calculated using [text](#DMC049_outflow)

```{math}
:label: DMC049_outflow

Q_{C\_DMC049}^{t} = Q_{C\_DMC044}^{t} - Q_{D\_DMC049\_DMCLOS}^{t}

```

Where:
- {math}`Q_{C\_DMC049}^{t}`: Outflow at node DMC049 for timestep {math}`t`
- {math}`Q_{D\_DMC049\_DMCLOS}^{t}`: Losses at node DMC049 for timestep {math}`t`

The fraction of NVWRRP discharge at node DMC049 ({math}`f_{DMC049}^{t}`) is calculated using the mass balance shown in [text](#DMC049_mass_balance)

```{math}
:label: DMC049_mass_balance

Q_{C\_DMC044}^{t}*f_{DMC044}^{t} = Q_{C\_DMC049}^{t}*f_{DMC049}^{t}

```
Where:
- {math}`f_{DMC049}^{t}`: Fraction of NVWRRP water at node DMC049 for timestep {math}`t`

{math}`f_{DMC049}^{t}` is calculated rearranging [text](#DMC049_mass_balance) into [text](#f_DMC049)

```{math}
:label: f_DMC049

f_{DMC049}^{t} = \frac{Q_{C\_DMC044}^{t}*f_{DMC044}^{t}}{Q_{C\_DMC049}^{t}}

```
The outflow for node DMC054 is calculated using [text](#DMC054_outflow)
```{math}
:label: DMC054_outflow

Q_{C\_DMC054}^{t} = Q_{C\_DMC049}^{t} - Q_{D\_DMC054\_DMCLOS}^{t} - Q_{D\_DMC054\_NMW004}^{t}

```

Where:
- {math}`Q_{C\_DMC054}^{t}`: Outflow at node DMC054 for timestep {math}`t`
- {math}`Q_{D\_DMC054\_DMCLOS}^{t}`: Losses at node DMC054 for timestep {math}`t`
- {math}`Q_{D\_DMC054\_NMW004}^{t}`: Diversion to Newman Wasteway for timestep {math}`t`

The fraction of NVWRRP discharge at node DMC054 ({math}`f_{DMC054}^{t}`) is calculated using the mass balance shown in [text](#DMC054_mass_balance)

```{math}
:label: DMC054_mass_balance

Q_{C\_DMC049}^{t}*f_{DMC049}^{t} = (Q_{C\_DMC054}^{t}+Q_{D\_DMC054\_NMW004}^{t})*f_{DMC054}^{t}

```
Where:
- {math}`f_{DMC054}^{t}`: Fraction of NVWRRP water at node DMC054 for timestep {math}`t`

{math}`f_{DMC054}^{t}` is calculated rearranging [text](#DMC054_mass_balance) into [text](#f_DMC054)

```{math}
:label: f_DMC054

f_{DMC054}^{t} = \frac{Q_{C\_DMC049}^{t}}{Q_{C\_DMC054}^{t}+Q_{D\_DMC054\_NMW004}^{t}}*f_{DMC049}^{t}

```

The outflow for node DMC058 is calculated using [text](#DMC058_outflow)

```{math}
:label: DMC058_outflow

Q_{C\_DMC058}^{t} = Q_{C\_DMC054}^{t} - Q_{D\_DMC058\_DMCLOS}^{t}

```

Where:
- {math}`Q_{C\_DMC058}^{t}`: Outflow at node DMC058 for timestep {math}`t`
- {math}`Q_{D\_DMC058\_DMCLOS}^{t}`: Losses at node DMC054 for timestep {math}`t`

The fraction of NVWRRP discharge at node DMC058 ({math}`f_{DMC058}^{t}`) is calculated using the mass balance shown in [text](#DMC058_mass_balance)

```{math}
:label: DMC058_mass_balance

Q_{C\_DMC054}^{t}*f_{DMC054}^{t} = Q_{C\_DMC058}^{t}*f_{DMC058}^{t}

```
Where:
- {math}`f_{DMC058}^{t}`: Fraction of NVWRRP water at node DMC058 for timestep {math}`t`

{math}`f_{DMC058}^{t}` is calculated rearranging [text](#DMC058_mass_balance) into [text](#f_DMC058)

```{math}
:label: f_DMC058

f_{DMC058}^{t} = \frac{Q_{C\_DMC054}^{t}}{Q_{C\_DMC058}^{t}}*f_{DMC054}^{t}

```

The outflow for node DMC064 is calculated using [text](#DMC064_outflow)

```{math}
:label: DMC064_outflow

Q_{C\_DMC064}^{t} = Q_{C\_DMC058}^{t} - Q_{D\_DMC064\_DMCLOS}^{t} - Q_{D\_DMC064\_71\_PA6}^{t}

```

Where:
- {math}`Q_{C\_DMC064}^{t}`: Outflow at node DMC058 for timestep {math}`t`
- {math}`Q_{D\_DMC064\_DMCLOS}^{t}`: Losses at node DMC054 for timestep {math}`t`
- {math}`Q_{D\_DMC064\_71\_PA6}^{t}`: Diversions to Davis WD, Foothill WD, Mustang WD, Quinto WD, Romero WD, and Centinella WD for timestep {math}`t`

The fraction of NVWRRP discharge at node DMC064 ({math}`f_{DMC064}^{t}`) is calculated using the mass balance shown in [text](#DMC064_mass_balance)

```{math}
:label: DMC064_mass_balance

Q_{C\_DMC058}^{t}*f_{DMC058}^{t} = (Q_{C\_DMC064}^{t}+Q_{D\_DMC064\_71\_PA6}^{t})*f_{DMC064}^{t}

```
Where:
- {math}`f_{DMC064}^{t}`: Fraction of NVWRRP water at node DMC064 for timestep {math}`t`

{math}`f_{DMC064}^{t}` is calculated rearranging [text](#DMC064_mass_balance) into [text](#f_DMC064)

```{math}
:label: f_DMC064

f_{DMC064}^{t} = \frac{Q_{C\_DMC058}^{t}}{Q_{C\_DMC064}^{t}+Q_{D\_DMC064\_71\_PA6}^{t}}*f_{DMC058}^{t}

```

At node DMC070, inflow from O'Neill Forebay brings water with a fraction of NVWRRP discharge different than {math}`f_{DMC064}`. Therefore, in addition to the water balance, a mass balance is needed to keep track of the fraction of NVWRRP discharge leaving the node. The flow leaving node DMC070 to O'Neill Forebay is calculated from the node water balance, as shown in [text](#DMC070_outflow), while the mass balance is presented in [text](#DMC070_mass_balance)

```{math}
:label: DMC070_outflow

Q_{D\_DMC070\_CAA069}^{t} &= Q_{C\_DMC064}^{t} + Q_{D\_CAA069\_DMC070}^{t} - Q_{D\_DMC070\_DMCLOS}^{t}\\ &- Q_{D\_DMC070\_VLW008}^{t} - Q_{D\_DMC070\_73\_PA1}^{t} - Q_{C\_DMC070}^{t}

```

Where:
- {math}`Q_{C\_DMC070}^{t}`: Outflow at node DMC070 for timestep {math}`t`
- {math}`Q_{D\_CAA069\_DMC070}^{t}`: Inflow at node DMC070 from O'Neill Forebay for timestep {math}`t`
- {math}`Q_{D\_DMC070\_DMCLOS}^{t}`: Losses at node DMC070 for timestep {math}`t`
- {math}`Q_{D\_DMC070\_VLW008}^{t}`: Diversion at node DMC070 to Volta Wasteway for timestep {math}`t`
- {math}`Q_{D\_DMC070\_73\_PA1}^{t}`: Diversion at node DMC070 to San Luis WD for timestep {math}`t`
- {math}`Q_{D\_DMC070\_CAA069}^{t}`: Diversion at node DMC070 to O'Neill Forebay for timestep {math}`t`

<br>

```{math}
:label: DMC070_mass_balance

Q_{C\_DMC064}^{t}*f_{DMC064}^{t} + Q_{D\_CAA069\_DMC070}^{t}*f_{CAA069}^{t} &= (Q_{D\_DMC070\_VLW008}^{t} + Q_{D\_DMC070\_73\_PA1}^{t}+ Q_{D\_DMC070\_CAA069}^{t} \\&+ Q_{C\_DMC070}^{t})*f_{DMC070}^{t}

```

Where:
- {math}`f_{CAA069}^{t}`: Fraction of NVWRRP discharge in O'Neill Forebay for timestep {math}`t`
- {math}`f_{DMC070}^{t}`: Fraction of NVWRRP discharge on flow leaving node DMC070 for timestep {math}`t`



For node CAA069 (O'Neill Forebay), although CALSIM represents it as a conveyance node, we will represent it as a reservoir node in our analysis. Therefore, we need to consider the survey's storage in the mass balance, as shown in [text](#CAA069_mass_balance). We will work until the assumption of constant storage in the Forebay.

```{math}
:label: CAA069_flow_balance

Q_{C\_CAA067}^{t} + Q_{C\_SLUIS}^{t} + Q_{D\_DMC070\_CAA069}^{t}&\\-Q_{D\_CAA069\_DMC070}^{t} - Q_{C\_CAA069}^{t} - Q_{D\_CAA069\_SLUIS}^{t}&=0
```

Where:
- {math}`Q_{C\_CAA067}^{t}`: Inflow from node CAA067 of the California Aqueduct for timestep {math}`t`
- {math}`Q_{C\_SLUIS}^{t}`: Inflow from San Luis Reservoir for timestep {math}`t`
- {math}`Q_{C\_CAA069}^{t}`: Outflow to node CAA071 of the California Aqueduct for timestep {math}`t`
- {math}`Q_{D\_CAA069\_SLUIS}^{t}`: Released flow to San Luis Reservoir for timestep {math}`t`


```{math}
:label: CAA069_mass_balance

Q_{C\_SLUIS}^{t}*f_{SLUIS}^{t} + Q_{D\_DMC070\_CAA069}^{t}*f_{DMC070}^{t}&\\-(Q_{D\_CAA069\_DMC070}^{t} - Q_{C\_CAA069}^{t} - Q_{D\_CAA069\_SLUIS}^{t})*f_{CAA069}^{t}&=\forall^{t}_{CAA069}\frac{d}{dt}f_{CAA069}^{t}
```

Where:
- {math}`f_{SLUIS}^{t}`: Fraction of NVWRRP discharge in San Luis Reservoir for timestep {math}`t`
- {math}`\forall_{CAA069}`: Storage volume in O'Neill Forebay
- {math}`\frac{d}{dt}f_{CAA069}^{t}`: Change in fraction of NVWRRP discharge in O'Neill Forebay for timestep {math}`t`

The derivatives can be approximated using backwards finite differences, as shown in [text](#CAA069_mass_balance_fd)

```{math}
:label: CAA069_mass_balance_fd

Q_{C\_SLUIS}^{t}*f_{SLUIS}^{t} + Q_{D\_DMC070\_CAA069}^{t}*f_{DMC070}^{t}&\\-(Q_{D\_CAA069\_DMC070}^{t} + Q_{C\_CAA069}^{t} + Q_{D\_CAA069\_SLUIS}^{t})*f_{CAA069}^{t}&=\forall^{t}_{CAA069}\frac{f_{CAA069}^{t}-f_{CAA069}^{t-1}}{\Delta t}
```
Where:
- {math}`f_{CAA069}^{t-1}`: : Fraction of NVWRRP discharge in O'Neill Forebay for timestep {math}`t-1`
- {math}`\Delta t`: Length of timestep

Rearranging [text](#CAA069_flow_balance) we can find the flow released from O'Neill Forebay to San Luis Reservoir for timestep {math}`t` ({math}`Q_{D\_CAA069\_SLUIS}^{t}`) as shown in [text](#D_CAA069_SLUIS)

```{math}
:label: D_CAA069_SLUIS

Q_{D\_CAA069\_SLUIS}^{t}= Q_{C\_CAA067}^{t} + Q_{C\_SLUIS}^{t} + Q_{D\_DMC070\_CAA069}^{t}&\\-Q_{D\_CAA069\_DMC070}^{t}- Q_{C\_CAA069}^{t} - \frac{\forall^{t}_{CAA069}-\forall^{t-1}_{CAA069}}{\Delta t}
```

In the same way as O'Neill Forebay, we can setup flow and mass balances for San Luis Reservoir, as shown in [text](#SLUIS_flow_balance_fd) and [text](#SLUIS_mass_balance_fd)

```{math}
:label: SLUIS_flow_balance_fd

Q_{I\_SLUIS}^{t} + Q_{D\_CAA069\_SLUIS}^{t} - Q_{D\_SLUIS\_PCH000}^{t} -Q_{C\_SLUIS}^{t} -Q_{E\_SLUIS}^{t} = \frac{\forall^{t}_{SLUIS}-\forall^{t-1}_{SLUIS}}{\Delta t}+\epsilon^{t}
```

Where
- {math}`Q_{I\_SLUIS}^{t}`: Runoff inflow to San Luis reservoir for timestep {math}`t`
- {math}`Q_{D\_SLUIS\_PCH000}^{t}`: Flow released from San Luis reservoir to Pacheco Tunnel for timestep {math}`t`
- {math}`Q_{E\_SLUIS}^{t}`: Flow of evaporated from San Luis reservoir  for timestep {math}`t`
- {math}`\forall^{t}_{SLUIS}`: Volume of effective storage in San Luis reservoir for timestep {math}`t`
- {math}`\forall^{t-1}_{SLUIS}`: Volume of effective storage in San Luis reservoir for timestep {math}`t-1`
- {math}`\epsilon^{t}`: Closure term for timestep {math}`t`

For the water balance for San Luis Reservoir, the assumption is that any deviations on inflow to the reservoir resulting from NVWRRP discharge or use of water stored in San Luis Reservoir by DPWD will result in a change in storage in San Luis Reservoir ({math}`\frac{\forall^{t}_{SLUIS}-\forall^{t-1}_{SLUIS}}{\Delta t}`), and will not affect flow released to Pacheco Tunnel ({math}`Q_{D\_SLUIS\_PCH000}^{t}`). Under these assumptions, we can estimate the amount of storage available in San Luis Reservoir at timestep {math}`t` by rearranging [text](#SLUIS_flow_balance_fd), as shown in [text](#SLUIS_storage_t)


```{math}
:label: SLUIS_storage_t

\forall^{t}_{SLUIS} = \forall^{t-1}_{SLUIS} + \Delta t*(Q_{I\_SLUIS}^{t} + Q_{D\_CAA069\_SLUIS}^{t} - Q_{D\_SLUIS\_PCH000}^{t} -Q_{C\_SLUIS}^{t} -Q_{E\_SLUIS}^{t}-\epsilon^{t})

```
For the mass balance, we assume that the fraction of NVWRRP discharge in the evaporation flows for San Luis Reservoir ({math}`Q_{E\_SLUIS}^{t}`) is zero.

```{math}
:label: SLUIS_mass_balance_fd

Q_{D\_CAA069\_SLUIS}^{t}*f_{CAA069}^{t} - f_{SLUIS}^{t}(Q_{D\_SLUIS\_PCH000}^{t} + Q_{C\_SLUIS}^{t}+\epsilon^{t}) &= \frac{d}{dt}(\forall^{t}_{SLUIS}*f_{SLUIS}^{t}) \\&= \forall^{t}_{SLUIS}*\frac{d}{dt}f_{SLUIS}^{t} + f_{SLUIS}^{t}*\frac{d}{dt}\forall^{t}_{SLUIS} \\&= f_{SLUIS}^{t}\frac{2\forall^{t}_{SLUIS}-\forall^{t-1}_{SLUIS}}{\Delta t} - \frac{\forall^{t}_{SLUIS}*f_{SLUIS}^{t-1}}{\Delta t}


```
Where:
- {math}`f_{SLUIS}^{t-1}`: Fraction of NVWRRP discharge in San Luis Reservoir for timestep {math}`t-1`

To solve for {math}`f_{DMC070}^{t}`, {math}`f_{CAA069}^{t}`, and {math}`f_{SLUIS}^{t}`, we need to solve equations [text](#DMC070_mass_balance), [text](#CAA069_mass_balance_fd_sim), and [text](#SLUIS_mass_balance_fd) simultaneously. Since these equations are linear, we will express them in matrix form {math}`\overline{\overline{A}}\overline{x}=\overline{b}`. To do this, we rearrange the equations as shown in [text](#matrix_form_0) and [text](#matrix_form)
```{math}
:label: matrix_form_0

(Q_{D\_DMC070\_VLW008}^{t} + Q_{D\_DMC070\_73\_PA1}^{t}+ Q_{D\_DMC070\_CAA069}^{t}+ Q_{C\_DMC070}^{t})*f_{DMC070}^{t} - Q_{D\_CAA069\_DMC070}^{t}*f_{CAA069}^{t} &= Q_{C\_DMC064}^{t}*f_{DMC064}^{t}

 Q_{D\_DMC070\_CAA069}^{t}*f_{DMC070}^{t} -(Q_{D\_CAA069\_DMC070}^{t} + Q_{C\_CAA069}^{t} + Q_{D\_CAA069\_SLUIS}^{t}+\frac{\forall^{t}_{CAA069}}{\Delta t})*f_{CAA069}^{t} + Q_{C\_SLUIS}^{t}*f_{SLUIS}^{t} &= -\forall^{t}_{CAA069}\frac{f_{CAA069}^{t-1}}{\Delta t}

Q_{D\_CAA069\_SLUIS}^{t}*f_{CAA069}^{t}- f_{SLUIS}^{t}(Q_{D\_SLUIS\_PCH000}^{t} + Q_{C\_SLUIS}^{t}+\epsilon^{t}+\frac{2\forall^{t}_{SLUIS}-\forall^{t-1}_{SLUIS}}{\Delta t}) &=  - \frac{\forall^{t}_{SLUIS}*f_{SLUIS}^{t-1}}{\Delta t}

  

```

```{math}
:label: matrix_form

\begin{pmatrix}
Q_{D\_DMC070\_VLW008}^{t} + Q_{D\_DMC070\_73\_PA1}^{t}+ Q_{D\_DMC070\_CAA069}^{t}+ Q_{C\_DMC070}^{t} & Q_{D\_CAA069\_DMC070}^{t} & 0\\
Q_{D\_DMC070\_CAA069}^{t} & -(Q_{D\_CAA069\_DMC070}^{t} + Q_{C\_CAA069}^{t} + Q_{D\_CAA069\_SLUIS}^{t}+\frac{\forall^{t}_{CAA069}}{\Delta t}) & Q_{C\_SLUIS}^{t}\\
0 & Q_{D\_CAA069\_SLUIS}^{t} & -(Q_{D\_SLUIS\_PCH000}^{t} + Q_{C\_SLUIS}^{t}+\epsilon^{t}+\frac{2\forall^{t}_{SLUIS}-\forall^{t-1}_{SLUIS}}{\Delta t})
\end{pmatrix}*\begin{pmatrix}
f_{DMC070}^{t}\\
f_{CAA069}^{t}\\
f_{SLUIS}^{t}
\end{pmatrix} = \begin{pmatrix}
Q_{C\_DMC064}^{t}*f_{DMC064}^{t} \\
-\forall^{t}_{CAA069}\frac{f_{CAA069}^{t-1}}{\Delta t} \\
- \frac{\forall^{t}_{SLUIS}*f_{SLUIS}^{t-1}}{\Delta t} \\
\end{pmatrix}

```

Rearranging, we can find the fractions at each node, as shown in 
```{math}
:label: f_DMC070_CAA069_SLUIS

\begin{pmatrix}
f_{DMC070}^{t}\\
f_{CAA069}^{t}\\
f_{SLUIS}^{t}
\end{pmatrix} = \begin{pmatrix}
Q_{D\_DMC070\_VLW008}^{t} + Q_{D\_DMC070\_73\_PA1}^{t}+ Q_{D\_DMC070\_CAA069}^{t}+ Q_{C\_DMC070}^{t} & Q_{D\_CAA069\_DMC070}^{t} & 0\\
Q_{D\_DMC070\_CAA069}^{t} & -(Q_{D\_CAA069\_DMC070}^{t} + Q_{C\_CAA069}^{t} + Q_{D\_CAA069\_SLUIS}^{t}+\frac{\forall^{t}_{CAA069}}{\Delta t}) & Q_{C\_SLUIS}^{t}\\
0 & Q_{D\_CAA069\_SLUIS}^{t} & -(Q_{D\_SLUIS\_PCH000}^{t} + Q_{C\_SLUIS}^{t}+\epsilon^{t}+\frac{2\forall^{t}_{SLUIS}-\forall^{t-1}_{SLUIS}}{\Delta t})
\end{pmatrix}^{-1}*\begin{pmatrix}
Q_{C\_DMC064}^{t}*f_{DMC064}^{t} \\
-\forall^{t}_{CAA069}\frac{f_{CAA069}^{t-1}}{\Delta t} \\
- \frac{\forall^{t}_{SLUIS}*f_{SLUIS}^{t-1}}{\Delta t} \\
\end{pmatrix}
```

## Modelling Assumptions
- Longitudinal dispersion is negligible.
- Travel times in conveyance nodes are much shorter than the length of the simulation timestep.
- Storage is only possible in nodes of type "Reservoir" (stage stays constant in conveyance canals).
- Derivatives will be approximated using backwards finite differences.
- Original CALSIM losses don't change with deviations in flows derived from the NVWRRP.
- Deviations in flow coming into O'Neill Forebay as a consequence fo the NVWRRP don't result in changes in storage at O'Neill Forebay. Rather, they are passed to San Luis Reservoir, where the storage is affected by the NVWRRP operation.
- NVWRRP discharge can be stored in San Luis Reservoir, but it's only delivered to DPWD through node DMC044
- Evaporation at San Luis Reservoir concentrates constituents. Therefore, it is conservative to assume that the fraction of NVWRRP in evaporated water is 0.
- Storage and surface area remain fixed at O'Neill Forebay.
- Losses at O'Neill Forebay are already considered in CALSIM flows. Therefore, evaporation in O'Neill Forebay won't be explicitly considered.
- Additional flows resulting from the NVWRRP do not result in additional conveyance losses.
- Since the DMC is lined, conveyance losses consist mainly in evaporation. Since evaporation concentrates constituents, we assign a value of 0 to the fraction of NVWRRP water in conveyance losses as a conservative assumption.